# Sliders/Interactive Elements Tutorial

To create interactive sliders, we use the python package bokeh. You can install the necessary packages for bokeh at this [link](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html).

We start by defining functions for the mathematics of the plot, that will update based on the parameters. We then give an output file and and a set of variables that we will use to create the intiial visualization. Next, we will create sliders for the variables we wish to change.

The tricky part is that to update the graph, we rewrite the math in JavaScript, that will update the html as we move the slider. Below, we can see an example of this in action. This is a slider for changing the potential, and the resulting effect on position. 

In [ ]:
import numpy as np
def g(u, v, d):
        ''' g function (equation 3.12 in the davies paper) '''
        R = np.sqrt(u**2 + v**2 + d**2)
        return 1/(2*np.pi) * np.arctan(u*v/(d*R))

def phi_rect(x, y, d, Vg, L, R, B, T):
    ''' phi equation (3.11, davies paper)'''
    return Vg*(g(x-L, y-B, d) + g(x-L, T-y, d) + g(R-x, y-B, d) + g(R-x, T-y, d))

def GatePotential(gate_width, dot_width, gap, V1, VP, V2, VD, V0, depth, height, inf, x, y=0):
    ''' returns the potential for a gate structure '''
    out = 0
    # first rectangle is T=+inf, B=-inf, L=-inf, R=0
    out += phi_rect(x, y, depth, V0, -inf, 0, -inf, inf)
    # V1: L = gap, R = gate_width+gap
    out += phi_rect(x, y, depth, V1, gap, gate_width+gap, -height/2, height/2)
    # Vp: L = gate_width+2gap, R = gate_width+2gap+dot_width
    out += phi_rect(x, y, depth, VP, gate_width + 2*gap, gate_width + 2*gap+dot_width, -height/2, height/2)
    #V2: L = gate_width+3gap+dot_width, R = 2*gate_width+3gap+dot_width
    out += phi_rect(x, y, depth, V2, gate_width + 3*gap + dot_width, 2*gate_width + 3*gap + dot_width, -height/2, height/2)
    #Vd: L=2*gate_width+4gap+dot_width, R=inf
    out += phi_rect(x,y,depth,VD,2*gate_width + 4*gap + dot_width, inf, -inf, inf)
    # return the potential accounting for all gates
    return out

In [ ]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiLine
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html
import IPython

# needed if testing via show() in some browser installations
# (not this exact location, of course; but something the
#  browser can access).
# comment out for jupyter book implementation
# output_file("/Users/clinic/Desktop/HRL html/temp.html")
xs = np.linspace(-300e-9, 500e-9, 10000)
ys = np.zeros_like(xs)
GATE_WIDTH = 50e-9
DOT_WIDTH = 90e-9
GAP = 10e-9
V1 = 1e-3
VP = -1e-3
V2= 1e-3
VD = -1e-5
V0 = 0
DEPTH = 50e-9
HEIGHT = 1
INF = 1e5


for i, x in enumerate(xs):
    ys[i] = GatePotential(GATE_WIDTH,DOT_WIDTH, GAP, V1, VP, V2, VD,V0, DEPTH,HEIGHT, INF,x )


source = ColumnDataSource(data=dict(x = xs, y = ys))

plot = figure(y_range=(-9e-4, 14e-4), x_range=(-300e-9, 500e-9), width=800, height=800,
              x_axis_label=r"$$\text{X Position}$$",
              y_axis_label=r"$$\text{ Potential(V)}$$")

plot.line("x","y",source=source)


voltage_slider = Slider(start=-0.004, end=0.007, value= 0.001, step=0.001,format = '0.000', title=r"$$V_1$$")

callback = CustomJS(args=dict(source=source, GATE_WIDTH = GATE_WIDTH, DOT_WIDTH = DOT_WIDTH, GAP = GAP, V1 = voltage_slider, VP = VP, V2= V2, VD = VD, V0 = V0, DEPTH = DEPTH, HEIGHT = HEIGHT, INF = INF),
                    code="""
    const v1n = V1.value

    const x = source.data.x
    const y = []
    
    function g(u, v, d) {
        const R = Math.sqrt(u ** 2 + v ** 2 + d ** 2);
        return (1 / (2 * Math.PI)) * Math.atan((u * v) / (d * R));
    }

    function phi_rect(x, y, d, Vg, L, R, B, T) {
        return Vg * (g(x - L, y - B, d) + g(x - L, T - y, d) + g(R - x, y - B, d) + g(R - x, T - y, d));
    }

    function GatePotential(gate_width, dot_width, gap, V1, VP, V2, VD, V0, depth, height, inf, x, y = 0) {
        let out = 0;
        out += phi_rect(x, y, depth, V0, -inf, 0, -inf, inf);
        out += phi_rect(x, y, depth, V1, gap, gate_width + gap, -height / 2, height / 2);
        out += phi_rect(x, y, depth, VP, gate_width + 2 * gap, gate_width + 2 * gap + dot_width, -height / 2, height / 2);
        out += phi_rect(x, y, depth, V2, gate_width + 3 * gap + dot_width, 2 * gate_width + 3 * gap + dot_width, -height / 2, height / 2);
        out += phi_rect(x, y, depth, VD, 2 * gate_width + 4 * gap + dot_width, inf, -inf, inf);
        return out;
    }
    
    for (let i = 0; i < x.length; i++) {
        y[i] = GatePotential(GATE_WIDTH, DOT_WIDTH, GAP, v1n, VP, V2, VD, V0, DEPTH, HEIGHT, INF, x[i]);
    }

    source.data = {x, y}
""")

voltage_slider.js_on_change('value', callback)


html_repr = file_html(row(plot, column(voltage_slider)), CDN)
IPython.display.HTML(html_repr)
# debugging
# show(row(plot, column(voltage_slider)))